In [399]:
import numpy as np
import pandas as pd
from typing import List, Optional
#from caas_jupyter_tools import display_dataframe_to_user
from collections import defaultdict, deque

In [400]:
pd.set_option('display.max_rows', None)

In [401]:
# Input tensor shape: [N, H, W, C]
# Weight tensor shape: [R, S, C, K] 
# Output tensor shape: [N, P, Q, K]

def _output_size(I, K, P, S): 
    return int((I - K + 2*P)/(S)) + 1
    
N = 1 # batch size

H = 5 # activation height, activation width
W = 7
C = 1   # input channels <= strictly, grid_size (max PE size)
 
R = S = kernel_size = 4 # kernel height, kernel width
K = 1   # kernel channels, <= strictly, grid_size (max PE size)

# C and K don't need to be equal 

padding = 0
stride = 1

P = _output_size(H, R, padding, stride)
Q = _output_size(W, S, padding, stride)
num_patches = P * Q 

input_shape = (N, H, W, C)
weight_shape = (R, S, C, K)
output_shape = (N, P, Q, K)

In [402]:
output_shape

(1, 2, 4, 1)

In [403]:
N_idx, H_idx, W_idx, C_idx = np.indices((N, H, W, C))
activations = np.char.add(
    np.char.add(
        np.char.add(
            np.char.add('A', N_idx.astype(str)),
            H_idx.astype(str)
        ),
        W_idx.astype(str)
    ),
    C_idx.astype(str)
)

In [404]:
I_ker, J_ker, C_ker, K_ker = np.indices((R, S, C, K))
kernel = np.char.add(
    np.char.add(
        np.char.add(
            np.char.add('K', I_ker.astype(str)),
            J_ker.astype(str)
        ),
        C_ker.astype(str)
    ),
    K_ker.astype(str)
)

In [405]:
kernel.flatten()

array(['K0000', 'K0100', 'K0200', 'K0300', 'K1000', 'K1100', 'K1200',
       'K1300', 'K2000', 'K2100', 'K2200', 'K2300', 'K3000', 'K3100',
       'K3200', 'K3300'], dtype='<U85')

In [406]:
equations_matrix = [[[[None for _ in range(K)] 
                      for _ in range(Q)] 
                     for _ in range(P)] 
                    for _ in range(N)]
patches = []
inv_patches = []

for n in range(N):
    for i in range(0, H - R + 1, stride):
        for j in range(0, W - S + 1, stride):
            # Extract a patch from image n
            patch = activations[n, i:i+R, j:j+S, :]   # shape (R, S, C)
            print(i)
            print(j)
            print(patch)
            inv_patch = patch.transpose(1, 0, 2) # inverted: shape (S, R, C)
            patches.append(patch)
            inv_patches.append(inv_patch)
            
            for k_out in range(K):
                terms = []
                for a in range(R):
                    for b in range(S):
                        for c_idx in range(C):
                            # Each term: A{n}{i+a}{j+b}{c_idx} * K{a}{b}{c_idx}{k_out}
                            terms.append(f"{patch[a, b, c_idx]}*{kernel[a, b, c_idx, k_out]}")
                equation = " + ".join(terms)
                row_idx = i // stride
                col_idx = j // stride
                equations_matrix[n][row_idx][col_idx][k_out] = equation

0
0
[[['A0000']
  ['A0010']
  ['A0020']
  ['A0030']]

 [['A0100']
  ['A0110']
  ['A0120']
  ['A0130']]

 [['A0200']
  ['A0210']
  ['A0220']
  ['A0230']]

 [['A0300']
  ['A0310']
  ['A0320']
  ['A0330']]]
0
1
[[['A0010']
  ['A0020']
  ['A0030']
  ['A0040']]

 [['A0110']
  ['A0120']
  ['A0130']
  ['A0140']]

 [['A0210']
  ['A0220']
  ['A0230']
  ['A0240']]

 [['A0310']
  ['A0320']
  ['A0330']
  ['A0340']]]
0
2
[[['A0020']
  ['A0030']
  ['A0040']
  ['A0050']]

 [['A0120']
  ['A0130']
  ['A0140']
  ['A0150']]

 [['A0220']
  ['A0230']
  ['A0240']
  ['A0250']]

 [['A0320']
  ['A0330']
  ['A0340']
  ['A0350']]]
0
3
[[['A0030']
  ['A0040']
  ['A0050']
  ['A0060']]

 [['A0130']
  ['A0140']
  ['A0150']
  ['A0160']]

 [['A0230']
  ['A0240']
  ['A0250']
  ['A0260']]

 [['A0330']
  ['A0340']
  ['A0350']
  ['A0360']]]
1
0
[[['A0100']
  ['A0110']
  ['A0120']
  ['A0130']]

 [['A0200']
  ['A0210']
  ['A0220']
  ['A0230']]

 [['A0300']
  ['A0310']
  ['A0320']
  ['A0330']]

 [['A0400']
  ['A0410']
  ['A0

In [407]:
split_equations_dict = {}

for n in range(N):
    for row_id in range(len(equations_matrix[n])):
        for col_id in range(len(equations_matrix[n][row_id])):
            for k_out in range(K):
                equation = equations_matrix[n][row_id][col_id][k_out]
                # each part corresponding to a kernel row
                terms = equation.split(" + ")
                split_eq = []
                num_terms_per_split = S * C  # originally, each kernel row contributed S * C terms.
                for r_idx in range(R):
                    start = r_idx * num_terms_per_split
                    end = (r_idx + 1) * num_terms_per_split
                    partial_eq = " + ".join(terms[start:end])
                    
                    subterms = partial_eq.split(" + ")
                    if len(subterms) > 4:
                        # even more break the partial equation into groups of 4 terms.
                        num_groups = (len(subterms) + 3) // 4  
                        for group in range(num_groups):
                            sub_start = group * 4
                            sub_end = (group + 1) * 4
                            sub_eq = " + ".join(subterms[sub_start:sub_end])
                            key = f"C{n}{row_id}{col_id}{k_out}_{r_idx}_{group}"
                            split_equations_dict[sub_eq] = key
                    else:
                        key = f"C{n}{row_id}{col_id}{k_out}_{r_idx}"
                        split_equations_dict[partial_eq] = key

In [408]:
equations_matrix

[[[['A0000*K0000 + A0010*K0100 + A0020*K0200 + A0030*K0300 + A0100*K1000 + A0110*K1100 + A0120*K1200 + A0130*K1300 + A0200*K2000 + A0210*K2100 + A0220*K2200 + A0230*K2300 + A0300*K3000 + A0310*K3100 + A0320*K3200 + A0330*K3300'],
   ['A0010*K0000 + A0020*K0100 + A0030*K0200 + A0040*K0300 + A0110*K1000 + A0120*K1100 + A0130*K1200 + A0140*K1300 + A0210*K2000 + A0220*K2100 + A0230*K2200 + A0240*K2300 + A0310*K3000 + A0320*K3100 + A0330*K3200 + A0340*K3300'],
   ['A0020*K0000 + A0030*K0100 + A0040*K0200 + A0050*K0300 + A0120*K1000 + A0130*K1100 + A0140*K1200 + A0150*K1300 + A0220*K2000 + A0230*K2100 + A0240*K2200 + A0250*K2300 + A0320*K3000 + A0330*K3100 + A0340*K3200 + A0350*K3300'],
   ['A0030*K0000 + A0040*K0100 + A0050*K0200 + A0060*K0300 + A0130*K1000 + A0140*K1100 + A0150*K1200 + A0160*K1300 + A0230*K2000 + A0240*K2100 + A0250*K2200 + A0260*K2300 + A0330*K3000 + A0340*K3100 + A0350*K3200 + A0360*K3300']],
  [['A0100*K0000 + A0110*K0100 + A0120*K0200 + A0130*K0300 + A0200*K1000 + A021

In [409]:
split_equations_dict

{'A0000*K0000 + A0010*K0100 + A0020*K0200 + A0030*K0300': 'C0000_0',
 'A0100*K1000 + A0110*K1100 + A0120*K1200 + A0130*K1300': 'C0000_1',
 'A0200*K2000 + A0210*K2100 + A0220*K2200 + A0230*K2300': 'C0000_2',
 'A0300*K3000 + A0310*K3100 + A0320*K3200 + A0330*K3300': 'C0000_3',
 'A0010*K0000 + A0020*K0100 + A0030*K0200 + A0040*K0300': 'C0010_0',
 'A0110*K1000 + A0120*K1100 + A0130*K1200 + A0140*K1300': 'C0010_1',
 'A0210*K2000 + A0220*K2100 + A0230*K2200 + A0240*K2300': 'C0010_2',
 'A0310*K3000 + A0320*K3100 + A0330*K3200 + A0340*K3300': 'C0010_3',
 'A0020*K0000 + A0030*K0100 + A0040*K0200 + A0050*K0300': 'C0020_0',
 'A0120*K1000 + A0130*K1100 + A0140*K1200 + A0150*K1300': 'C0020_1',
 'A0220*K2000 + A0230*K2100 + A0240*K2200 + A0250*K2300': 'C0020_2',
 'A0320*K3000 + A0330*K3100 + A0340*K3200 + A0350*K3300': 'C0020_3',
 'A0030*K0000 + A0040*K0100 + A0050*K0200 + A0060*K0300': 'C0030_0',
 'A0130*K1000 + A0140*K1100 + A0150*K1200 + A0160*K1300': 'C0030_1',
 'A0230*K2000 + A0240*K2100 + A025

In [410]:
patches = np.array(patches)
inv_patches = np.array(inv_patches)

In [411]:
patches.shape

(8, 4, 4, 1)

In [412]:
kernel.flatten()

array(['K0000', 'K0100', 'K0200', 'K0300', 'K1000', 'K1100', 'K1200',
       'K1300', 'K2000', 'K2100', 'K2200', 'K2300', 'K3000', 'K3100',
       'K3200', 'K3300'], dtype='<U85')

In [413]:
# # ============================================================
# # Routing + Accumulation Structures
# # ============================================================
# # Each output pixel (n, p, q, k) gets its own FIFO
# acc_fifos = defaultdict(lambda: deque())  
# scratchpad = {}

# def route_and_accumulate(cycle, pe_outputs):
#     """
#     Route PE outputs (from bottom row) into correct accumulator FIFOs.
#     Flush to scratchpad when a full sum is collected.
#     """
#     for j, psum in enumerate(pe_outputs):
#         if psum == "0" or psum.strip() == "":
#             continue

#         # ----------------------------
#         # ROUTING FUNCTION
#         # ----------------------------
#         # Example: bottom row index j → output pixel (row, col)
#         # (You can refine mapping logic depending on stride/padding)
#         out_row = cycle // Q
#         out_col = j
#         dest = (0, out_row, out_col, 0)  # (n, p, q, k)

#         # ----------------------------
#         # CROSSBAR STAGE
#         # ----------------------------
#         # This is where multiple PE outputs in the same cycle
#         # are arbitrated and sent to the correct FIFO
#         acc_fifos[dest].append(psum)

#         # ----------------------------
#         # FLUSH TO SCRATCHPAD
#         # ----------------------------
#         # Once we have R*S*C terms for (n,p,q,k), flush sum
#         if len(acc_fifos[dest]) == R * S * C:
#             final_sum = " + ".join(acc_fifos[dest])
#             scratchpad[dest] = final_sum
#             acc_fifos[dest].clear()
#             print(f"[Cycle {cycle}] Flushed {dest} → {final_sum}")

In [414]:
class PE:
    # links: [0: left, 1: up, 2: right, 3: down]
    def __init__(self, links: Optional[List['PE']] = None):
        if links is None:
            links = [None, None, None, None]
        self.links = links
        self.activation = "0"
        self.weight = '--'
        self.accumulation: str = "0"
    
    def _input(self, activation: str):
        self.activation = activation
    
    def _weight(self, weight: str):
        self.weight = weight
    
    def shift(self, shift_direction: int, activation_flag: bool):
        neighbor = self.links[shift_direction]
        if neighbor is not None:
            if activation_flag:
                neighbor._input(self.activation)
            else:
                neighbor._weight(self.weight)

class SystolicArray:
    def __init__(self, size: int):
        self.size = size
        self.buffer = []  
        self.array = self._setup_array()
    
    def _setup_array(self):
        array = [[PE() for _ in range(self.size)] for _ in range(self.size)]
        for i in range(self.size):
            for j in range(self.size):
                left = array[i][j-1] if j > 0 else None
                up = array[i-1][j] if i > 0 else None
                right = array[i][j+1] if j < self.size - 1 else None
                down = array[i+1][j] if i < self.size - 1 else None
                array[i][j].links = [left, up, right, down]
        return array

    def print_array(self):
        for i in range(self.size):
            row_parts = []
            for pe in self.array[i]:
                act_str = pe.activation.ljust(12)
                weight_str = pe.weight.ljust(12)
                
                if pe.accumulation.strip() == "0":
                    parts = self.size * ["0"]
                else:
                    parts = [p.strip() for p in pe.accumulation.split('+')]
                    while len(parts) < kernel_size:
                        parts.append("0")
                    parts = parts[:4]
                parts = [p.ljust(12) for p in parts]
                acc_str = " + ".join(parts)
                
                row_parts.append(f"[A:{act_str} | W:{weight_str} | Acc:{acc_str}]")
            print(" | ".join(row_parts))

    def cycle(self, new_activations: Optional[List[str]] = None):
        if new_activations:
            for i in range(self.size):
                #  shift activations rightward
                for j in range(self.size - 1, 0, -1):
                    self.array[i][j].activation = self.array[i][j-1].activation
                # shift the new activation into column 0.
                self.array[i][0].activation = new_activations[i]
        
        bottom_row = [self.array[self.size - 1][j].accumulation for j in range(self.size)]
        self.buffer.append(bottom_row)
        # Look to get the equations
        
        for j in range(self.size):
            # save previous cycle's accumulations for column j
            prev_acc = [self.array[i][j].accumulation for i in range(self.size)]
            
            # Update the accumulation in row 0 for column j.
            cell0 = self.array[0][j]
            prod0 = f"{cell0.activation}*{cell0.weight}" if cell0.activation not in ["0", "--"] else "0"
            cell0.accumulation = prod0
            
            for i in range(1, self.size):
                cell = self.array[i][j]
                prod = f"{cell.activation}*{cell.weight}" if cell.activation not in ["0", "--"] else "0"
                inherited = prev_acc[i-1]
                if inherited == "0":
                    new_acc = prod
                else:
                    new_acc = inherited + (f" + {prod}" if prod != "0" else "")
                cell.accumulation = new_acc if new_acc != "" else "0"

In [415]:
class Conv2dSimulator:
    def __init__(self, grid_size: int):
        self.input_buffers = [
            [],              
            ["0"],           
            ["0", "0"],      
            ["0", "0", "0"] 
        ]
        
        self.grid_size = grid_size
        self.systolic_array = SystolicArray(grid_size)
        self.outputs = [] 
        self.weight_update_flag = False
        self.kernel_update_iter = 0
        self.total_cycles = 1
        
        self.cycles_per_weight_frame = P * Q + 6
        self.current_cycle_count = 0
        
        self.current_kernel = None 
        self.current_kernel_index = 0  
        self.flat_patch = None
        self.complete = 0 

        #super().__init__(grid_size)
        
        # PSUM FIFO per output pixel: key = (n, p, q, k_out)
        self.acc_fifo = defaultdict(list)
        
        # keep track of how many contributions each PSUM has seen
        self.psum_count = defaultdict(int)
        
        # Total contributions required per output pixel
        self.rsc_per_output = R * S * C  # rows * cols * channels
        
        # Scratchpad for final outputs (simulation of memory writeback)
        self.scratchpad = np.full(output_shape, None, dtype=object)


    def add_patches(self, patches: List[np.ndarray]):     
        flat_patch = []
        if (C + K) > 2: 
            for h in range(patches.shape[1]):       # patches.shape[1] = H
                for w in range(patches.shape[2]):     # patches.shape[2] = W
                    for n in range(patches.shape[0]):  # patches.shape[0] = N
                        for c in range(patches.shape[3]): # patches.shape[3] = C
                            flat_patch.append(patches[n, h, w, c])
        else: 
            for n in range(patches.shape[0]):  # patches.shape[0] = N
                for h in range(patches.shape[1]):       # patches.shape[1] = H
                    for w in range(patches.shape[2]):     # patches.shape[2] = W
                        for c in range(patches.shape[3]): # patches.shape[3] = C
                            flat_patch.append(patches[n, h, w, c])
        self.flat_patch = flat_patch

        counter = 0
        for i in range(0, len(flat_patch), self.grid_size):
            chunk = flat_patch[i:i + self.grid_size]
            if len(chunk) < self.grid_size:
                chunk.extend(["0"] * (self.grid_size - len(chunk)))
            for j in range(self.grid_size):
                self.input_buffers[j].append(chunk[j])

            if (counter == (P * Q - 1)): 
                counter = 0
                for j in range(self.grid_size):
                    self.input_buffers[j].extend(["0"]*6)
            else: counter += 1

    def add_kernel(self, kernel: np.ndarray):
        self.current_kernel = kernel  
        self.current_kernel_index = 0  # start at the beginning of this kernel frame
        self.switch_weights()

    def switch_weights(self):
        grid_size = self.grid_size
        total_needed = grid_size * grid_size

        to_return = 0
        if (C + K) > 2: 
            flat_weights = self.current_kernel.flatten().tolist()
        else: 
            flat_weights = self.current_kernel.T.flatten().tolist()

        total_weights = len(flat_weights)

        start = self.current_kernel_index
        end = start + total_needed

        # If there aren't enough weights left, take what remains and pad with zeros.
        if end > total_weights:
            selected = flat_weights[start:] + [0] * (end - total_weights)
            self.complete = 1
            self.current_kernel_index = 0
        else:
            selected = flat_weights[start:end]
            self.current_kernel_index += total_needed
            if self.current_kernel_index >= total_weights:
                self.complete = 1
                self.current_kernel_index = 0

        for i in range(grid_size):
            for j in range(grid_size):
                idx = i * grid_size + j
                self.systolic_array.array[i][j].weight = selected[idx]

        self.kernel_update_iter += 1
        self.weight_update_flag = True
        self.current_cycle_count = 0
        print(f"Switched weights. Kernel update iteration: {self.kernel_update_iter}")
        return to_return

    # Helper function: compute output pixel for a given PE and input indices
    def compute_output_pixel(self, n_idx, h_idx, w_idx, k_out, pe_i, pe_j):
        # output row / col according to hardware formula
        out_row = (h_idx - pe_i) // stride
        out_col = (w_idx - pe_j) // stride
        if 0 <= out_row < P and 0 <= out_col < Q:
            return n_idx, out_row, out_col, k_out
        else:
            return None  # invalid PSUM, discard

    # NEW: route computed PSUM term into FIFO
    def route_psum(self, term, n_idx, h_idx, w_idx, k_out, pe_i, pe_j):
        dest = self.compute_output_pixel(n_idx, h_idx, w_idx, k_out, pe_i, pe_j)
        if dest:
            self.acc_fifo[dest].append(term)
            self.psum_count[dest] += 1

            # If FIFO has enough contributions, flush to scratchpad
            if self.psum_count[dest] == self.rsc_per_output:
                final_sum = " + ".join(self.acc_fifo[dest])
                self.scratchpad[dest] = final_sum
                self.acc_fifo[dest].clear()
                self.psum_count[dest] = 0
                print(f"Flushed PSUM to scratchpad at {dest}: {final_sum}")

    # •	n, p, q, k_out: Output pixel coordinates (batch n, row p, col q, output channel k_out).
	# •	Depth X/Y: How many contributions are in the FIFO vs how many are required (R*S*C).
	# •	Contents: Actual partial products waiting to be accumulated.

    def print_fifo_status(self):
        print("=== FIFO Status ===")
        for key, fifo in self.acc_fifo.items():
            n, p, q, k_out = key
            print(f"PSUM[{n},{p},{q},{k_out}] | Depth {len(fifo)}/{self.rsc_per_output} | Contents: {list(fifo)}")
        print("==================")

    # Modify cycle() to include FIFO status printing
    def cycle_with_fifo_debug(self):
        new_activations = []
        status = 0
        for i in range(self.systolic_array.size):
            if self.input_buffers[i]:
                new_act = self.input_buffers[i].pop(0)
            else:
                new_act = "0"
            new_activations.append(new_act)

        print("Total Cycles: ", self.total_cycles)
        print("New activations:", new_activations)
        self.systolic_array.cycle(new_activations=new_activations)
        self.systolic_array.print_array()

        # ROUTING: map each PE accumulation to correct PSUM FIFO
        for i in range(self.systolic_array.size):
            for j in range(self.systolic_array.size):
                pe = self.systolic_array.array[i][j]
                if pe.accumulation != "0":
                    terms = pe.accumulation.split(" + ")
                    for term in terms:
                        # Retrieve indices from term string: e.g., "A0000*K0000"
                        act_part, ker_part = term.split("*")
                        n_idx = int(act_part[1])
                        h_idx = int(act_part[2])
                        w_idx = int(act_part[3])
                        c_idx = int(act_part[4])

                        k_out = int(ker_part[4])
                        ker_i = int(ker_part[1])
                        ker_j = int(ker_part[2])

                        self.route_psum(term, n_idx, h_idx, w_idx, k_out, ker_i, ker_j)

        # PRINT FIFO STATUS PER CYCLE
        self.print_fifo_status()

        # Store output in simulation outputs (for logging)
        translated_equations = []
        for n in range(N):
            for p in range(P):
                for q in range(Q):
                    for k_out in range(K):
                        psum = self.scratchpad[n, p, q, k_out]
                        translated_equations.append(psum if psum else 'Cxx_x')
        self.outputs.append(translated_equations)
        print("Output: ", translated_equations)

        self.current_cycle_count += 1
        if not self.complete and (self.current_cycle_count >= self.cycles_per_weight_frame):
            print(f"Completed {self.current_cycle_count} cycles for current weight frame. Switching weights.")
            status = self.switch_weights()

        if self.complete:
            print("Convolution Completed!")

        self.total_cycles += 1
        return translated_equations, status



    # def cycle(self):
    #     new_activations = []
    #     status = 0
    #     for i in range(self.systolic_array.size):
    #         if self.input_buffers[i]:
    #             new_act = self.input_buffers[i].pop(0)
    #         else:
    #             new_act = "0"
    #         new_activations.append(new_act)

    #     print("Total Cycles: ", self.total_cycles)
    #     print("New activations:", new_activations)
    #     self.systolic_array.cycle(new_activations=new_activations)
    #     self.systolic_array.print_array()

    #     # ROUTING: map each PE accumulation to correct PSUM FIFO
    #     for i in range(self.systolic_array.size):
    #         for j in range(self.systolic_array.size):
    #             pe = self.systolic_array.array[i][j]
    #             if pe.accumulation != "0":
    #                 terms = pe.accumulation.split(" + ")
    #                 for term in terms:
    #                     # Retrieve indices from term string: e.g., "A0000*K0000"
    #                     # Parse input activation and kernel indices
    #                     act_part, ker_part = term.split("*")
    #                     n_idx = int(act_part[1])
    #                     h_idx = int(act_part[2])
    #                     w_idx = int(act_part[3])
    #                     c_idx = int(act_part[4])
                        
    #                     k_out = int(ker_part[4])
    #                     ker_i = int(ker_part[1])
    #                     ker_j = int(ker_part[2])
                        
    #                     self.route_psum(term, n_idx, h_idx, w_idx, k_out, ker_i, ker_j)

    #     # Store output in simulation outputs (for logging)
    #     translated_equations = []
    #     for n in range(N):
    #         for p in range(P):
    #             for q in range(Q):
    #                 for k_out in range(K):
    #                     psum = self.scratchpad[n, p, q, k_out]
    #                     translated_equations.append(psum if psum else 'Cxx_x')
    #     self.outputs.append(translated_equations)
    #     print("Output: ", translated_equations)

    #     self.current_cycle_count += 1
    #     if not self.complete and (self.current_cycle_count >= self.cycles_per_weight_frame):
    #         print(f"Completed {self.current_cycle_count} cycles for current weight frame. Switching weights.")
    #         status = self.switch_weights()

    #     if self.complete:
    #         print("Convolution Completed!")

    #     self.total_cycles += 1
    #     return translated_equations, status

In [416]:
kernel.shape

(4, 4, 1, 1)

In [417]:
# conv_sim = Conv2dSimulator(kernel_size)
# conv_sim.add_patches(patches)
# conv_sim.add_kernel(kernel)
# print(conv_sim.input_buffers[0])

# for cycle_num in range(200):
#     out, status = conv_sim.cycle()
#     print()

conv_sim = Conv2dSimulator(kernel_size)
conv_sim.add_patches(patches)
conv_sim.add_kernel(kernel)
# Bind this new cycle method to your simulator
conv_sim.cycle = cycle_with_fifo_debug.__get__(conv_sim)


for cycle_num in range(200):
    out, status = conv_sim.cycle()
    print()

    # Keep the same flow of activations, we need to check if we can move the weights in (flat similar to the log file), in a upper-left triangle way

Switched weights. Kernel update iteration: 1
Total Cycles:  1
New activations: [np.str_('A0000'), '0', '0', '0']
[A:A0000        | W:K0000        | Acc:A0000*K0000  + 0            + 0            + 0           ] | [A:0            | W:K1000        | Acc:0            + 0            + 0            + 0           ] | [A:0            | W:K2000        | Acc:0            + 0            + 0            + 0           ] | [A:0            | W:K3000        | Acc:0            + 0            + 0            + 0           ]
[A:0            | W:K0100        | Acc:0            + 0            + 0            + 0           ] | [A:0            | W:K1100        | Acc:0            + 0            + 0            + 0           ] | [A:0            | W:K2100        | Acc:0            + 0            + 0            + 0           ] | [A:0            | W:K3100        | Acc:0            + 0            + 0            + 0           ]
[A:0            | W:K0200        | Acc:0            + 0            + 0            + 0      

In [418]:
patches[4]

array([[['A0100'],
        ['A0110'],
        ['A0120'],
        ['A0130']],

       [['A0200'],
        ['A0210'],
        ['A0220'],
        ['A0230']],

       [['A0300'],
        ['A0310'],
        ['A0320'],
        ['A0330']],

       [['A0400'],
        ['A0410'],
        ['A0420'],
        ['A0430']]], dtype='<U85')

In [419]:
pd.DataFrame(conv_sim.outputs)

,0,1,2,3,4,5,6,7
0,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x
1,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x
2,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x
3,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x
4,A0000*K0000 + A0000*K0000 + A0010*K0100 + A010...,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x,Cxx_x
5,A0000*K0000 + A0000*K0000 + A0010*K0100 + A010...,Cxx_x,Cxx_x,Cxx_x,A0100*K0000 + A0100*K0000 + A0110*K0100 + A020...,Cxx_x,Cxx_x,Cxx_x
6,A0000*K0000 + A0000*K0000 + A0010*K0100 + A010...,Cxx_x,Cxx_x,Cxx_x,A0100*K0000 + A0100*K0000 + A0110*K0100 + A020...,Cxx_x,Cxx_x,Cxx_x
7,A0120*K1200 + A0200*K2000 + A0210*K2100 + A010...,Cxx_x,Cxx_x,Cxx_x,A0100*K0000 + A0100*K0000 + A0110*K0100 + A020...,Cxx_x,Cxx_x,Cxx_x
8,A0120*K1200 + A0200*K2000 + A0210*K2100 + A010...,A0010*K0000 + A0010*K0000 + A0020*K0100 + A011...,Cxx_x,Cxx_x,A0100*K0000 + A0100*K0000 + A0110*K0100 + A020...,Cxx_x,Cxx_x,Cxx_x
9,A0120*K1200 + A0200*K2000 + A0210*K2100 + A010...,A0010*K0000 + A0010*K0000 + A0020*K0100 + A011...,Cxx_x,Cxx_x,A0100*K0000 + A0100*K0000 + A0110*K0100 + A020...,A0110*K0000 + A0110*K0000 + A0120*K0100 + A021...,Cxx_x,Cxx_x


In [420]:
# INCOMPLETE!!  
# If PE(i,j) has seen the last input channel and last kernel row/column contribution for its assigned output,
# then mark its PSUM as final → write back to scratchpad.

def route_time(T, activation_size, output_size, kernel_size, stride, delay):
    routing = {}
    for k in range(kernel_size):
        remainder = T - kernel_size - (delay * k)
        if remainder < 0 or remainder % kernel_size != 0:
            routing[k] = None
            continue
        
        Q = remainder // kernel_size
        i = Q // output_size
        j = Q % output_size
        
        if i < output_size and j < output_size:
            routing[k] = (i, j)
        else:
            routing[k] = None
    return routing

for i in range(20): 
    print(i, route_time(0, H, P, R, stride, 2))

0 {0: None, 1: None, 2: None, 3: None}
1 {0: None, 1: None, 2: None, 3: None}
2 {0: None, 1: None, 2: None, 3: None}
3 {0: None, 1: None, 2: None, 3: None}
4 {0: None, 1: None, 2: None, 3: None}
5 {0: None, 1: None, 2: None, 3: None}
6 {0: None, 1: None, 2: None, 3: None}
7 {0: None, 1: None, 2: None, 3: None}
8 {0: None, 1: None, 2: None, 3: None}
9 {0: None, 1: None, 2: None, 3: None}
10 {0: None, 1: None, 2: None, 3: None}
11 {0: None, 1: None, 2: None, 3: None}
12 {0: None, 1: None, 2: None, 3: None}
13 {0: None, 1: None, 2: None, 3: None}
14 {0: None, 1: None, 2: None, 3: None}
15 {0: None, 1: None, 2: None, 3: None}
16 {0: None, 1: None, 2: None, 3: None}
17 {0: None, 1: None, 2: None, 3: None}
18 {0: None, 1: None, 2: None, 3: None}
19 {0: None, 1: None, 2: None, 3: None}


In [421]:
patches[0]

array([[['A0000'],
        ['A0010'],
        ['A0020'],
        ['A0030']],

       [['A0100'],
        ['A0110'],
        ['A0120'],
        ['A0130']],

       [['A0200'],
        ['A0210'],
        ['A0220'],
        ['A0230']],

       [['A0300'],
        ['A0310'],
        ['A0320'],
        ['A0330']]], dtype='<U85')

In [422]:
patches[1]

array([[['A0010'],
        ['A0020'],
        ['A0030'],
        ['A0040']],

       [['A0110'],
        ['A0120'],
        ['A0130'],
        ['A0140']],

       [['A0210'],
        ['A0220'],
        ['A0230'],
        ['A0240']],

       [['A0310'],
        ['A0320'],
        ['A0330'],
        ['A0340']]], dtype='<U85')